<a href="https://colab.research.google.com/github/sergejsrk/SparqlDatasetAnalysis/blob/main/Covid19_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
#atlasam visas klases kopa ar tam piederoso instancu skaitu
sparql_query_all_classes="""
SELECT ?c (count(?x) as ?cx) where 
{?x a ?c}   
GROUP BY ?c 
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx) 
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes}, 
                      headers={'Accept': 'application/json'})
i=0  
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

sparql_query_class_connectivity="""
SELECT ?c (count(?x) as ?cx) where 
{?x a ?c}   
GROUP BY ?c 
HAVING( count(?x) >= 1 )
ORDER BY desc(?cx) 
#limit 10000
"""

r = requests.get( "https://covidontheweb.inria.fr/sparql",
                      params={'query':sparql_query_all_classes}, 
                      headers={'Accept': 'application/json'})
i=0  
for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      # onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      # self.classes.append( onto_elem )
      i+=1
print(i)

7635


In [ ]:
# Table output experiments

from IPython.display import HTML, display

def display_table(data):
    html = """<table border="1" >
              <thead>
              <tr>
                <th>URI</th>
                <th>Instance Count</th>
              </tr>              
              </thead>
           """
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td width=\"80%%\">%s</td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

data = [["really long class name  with a full uri on top ",2],["short name",6]]
display_table(data)

URI,Instance Count
really long class name with a full uri on top,2
short name,6


In [ ]:
import requests

# url = "https://covidontheweb.inria.fr/sparql"
# sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"

# r = requests.get(url,
#                  params={'query':sparl_query_text}, 
#                  headers={'Accept': 'application/json'})
# print(r.content)
# print(f"Response: {r.json()}")
# res_json = r.json()

# print(res_json["results"]["bindings"])

# for e in res_json["results"]["bindings"]:
#   print(e["c"]["value"] + " " + e["cx"]["value"])

class DataSet:
  def __init__(self, name, endpoint_url):
    self.name = name
    self.endpoint_url = endpoint_url
    self.classes = []
    # self.properties = []
  
  def get_classes(self):
    sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
    r = requests.get( self.endpoint_url,
                      params={'query':sparl_query_text}, 
                      headers={'Accept': 'application/json'})
    
    for e in r.json()["results"]["bindings"]:
      # print(e["c"]["value"] + " " + e["cx"]["value"])
      onto_elem = OntologyElement(url = e["c"]["value"], instance_count = e["cx"]["value"])
      self.classes.append( onto_elem )

  def print_classes(self, min_amount_of_instances, class_name):
    for c in self.classes:
      if len(class_name) == 0:
        print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
               c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name in c.url and len(class_name) > 0 : 
          print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
                c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

  def create_list_of_classes(self, min_amount_of_instances, class_name_substr):
    #if class_name_patern has len > 0 we show only classes whose name contains class_name_substr
    result = []
    for c in self.classes:
      if int(c.instance_count) < int(min_amount_of_instances):
        break

      if len(class_name_substr) == 0:
        result.append(c)
        # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
        #        c.short_name_fragment  + "\t" + c.namespace)
      else:
        if class_name_substr in c.url and len(class_name_substr) > 0 : 
          result.append(c)
          # print( c.url + "\t" + c.instance_count + "\t" + c.short_name + "\t" + 
          #       c.short_name_fragment  + "\t" + c.namespace)
      if int(c.instance_count) < int(min_amount_of_instances):
        break

    return result

  def show_used_namespaces(self):
    namespace_map = {}
    for c in self.classes:
      if c.namespace in namespace_map:
        namespace_map[c.namespace] = namespace_map[c.namespace] + 1
      else:
        namespace_map[c.namespace] = 1
    count1 = 0
    for e in namespace_map:
      print(str(e) + "\t" + str(namespace_map[e]))
      count1 = count1 + namespace_map[e]
      
    print(count1)
  
  def is_class_present_in_ds(self, class_short_name):
    for c in self.classes:
      if class_short_name in c.url:
        return True

    return False

  def get_classes_from_ds(self, class_short_name):
    res = []
    for c in self.classes:
      if class_short_name in c.url:
        res.append(c)

    return res

  def collect_connectivity(self):
    print("Collecting connectivity ...")
    i=0
    for c in self.classes:    
      print("Collecting connectivity for " + c.url + " " + str(i) + " / " + str(len(self.classes))  )
      classURI = c.url
      s = """
      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <""" + classURI +""">.
      }
      """
      print(s)

      # sparl_query_text = " select ?c (count(?x) as ?cx) where {?x a ?c}   group by ?c   order by desc(?cx) limit 1000"
      r = requests.get( self.endpoint_url,
                        params={'query':s}, 
                        headers={'Accept': 'application/json'})
        
      for e in r.json()["results"]["bindings"]:
          c.connectivity = e["expr_COUNT"]["value"]
          print(e["expr_COUNT"]["value"])
      i+=1

def show_class_list_as_html_table(class_list, ds_name, ds_url):
  html = f"""<table border="1" >
            <caption> List of classes in {ds_name} ({ds_url}) </caption>
            <thead>
            <tr>
              <th>URI</th>
              <th>Short Name</th>
              <th>Instance Count</th>
              <th>Connectivity</th>
            </tr>              
            </thead>
          """
  for c in class_list:
      html += "<tr>"      
      html += "<td width=\"60%%\">%s</td>"%(c.url)
      html += "<td >%s</td>"%(c.short_name_fragment if len(c.short_name_fragment) > 0 else c.short_name )
      html += "<td >%s</td>"%(c.instance_count)
      html += "<td >%s</td>"%(c.connectivity)
      html += "</tr>"
  html += "</table>"
  display(HTML(html))


class OntologyElement:
  def __init__(self, url, instance_count):
    self.url = url
    self.instance_count = instance_count
    self.short_name = ""
    self.short_name_fragment = ""
    self.prefix = ""
    self.namespace = ""
    self.connectivity = 0
    

    if self.url.rfind("/") != -1:
      self.short_name = self.url[self.url.rfind("/") + 1:]  
      self.namespace = self.url[:self.url.rfind("/") ]  
    if self.short_name.find("#") != -1:
      self.short_name_fragment = self.short_name[self.short_name.find("#") + 1:]  
      self.namespace = self.url[:self.url.rfind("#") ]  
    
    # self.short_name = self.url[self.url.rfind("/"):]
    # self.short_name_fragment = self.short_name[self.short_name.find("#"):]


    
  

In [ ]:
ds1 = DataSet("covid19ontheweb", "https://covidontheweb.inria.fr/sparql")
ds1.get_classes()
ds1.collect_connectivity()
#ds1.show_used_namespaces()
print()
#ds1.print_classes(10,"")
show_class_list_as_html_table( ds1.create_list_of_classes(10,""), ds1.name, ds1.endpoint_url )

Streaming output truncated to the last 5000 lines.
        ?Drug a <http://dbpedia.org/resource/Number>.
      }
      
12493

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.wikidata.org/entity/Q186081>.
      }
      
20784

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.wikidata.org/entity/Q614112>.
      }
      
20784

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.wikidata.org/entity/Q937228>.
      }
      
20784

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.wikidata.org/entity/Q577>.
      }
      
20784

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (C

URI,Short Name,Instance Count,Connectivity
http://www.w3.org/ns/oa#Annotation,Annotation,74801097,2
http://www.w3.org/ns/oa#TextQuoteSelector,TextQuoteSelector,74801089,74801089
http://www.w3.org/ns/prov#Entity,Entity,74285514,19
http://www.w3.org/ns/oa#TextPositionSelector,TextPositionSelector,74285499,74285499
http://purl.obolibrary.org/obo/GO_0044419,GO_0044419,15807284,0
http://purl.obolibrary.org/obo/CARO_0010004,CARO_0010004,15423409,47421852
http://ns.inria.fr/wasabi/ontology/Song,Song,4188782,9
http://purl.org/spar/fabio/Work,Work,1226449,1804915
http://purl.org/spar/fabio/JournalArticle,JournalArticle,1124956,1658499
http://schema.org/Observation,Observation,658177,658177


In [ ]:
ds2 = DataSet("tweetscov19", "https://data.gesis.org/tweetscov19/sparql")
ds2.get_classes()
ds2.collect_connectivity()
# ds2.show_used_namespaces()
print()
# ds2.print_classes(10,"")
show_class_list_as_html_table( ds2.create_list_of_classes(10,""), ds2.name, ds2.endpoint_url )


Streaming output truncated to the last 5000 lines.
        ?Drug a <http://dbpedia.org/class/yago/WikicatAmericanProsecutors>.
      }
      
3816

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://dbpedia.org/class/yago/Attribute100024264>.
      }
      
212

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://dbpedia.org/class/yago/Difficulty114408086>.
      }
      
212

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://dbpedia.org/class/yago/GroupAction101080366>.
      }
      
212

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://dbpedia.org/class/yago/Homicide100220023>.
      }
      
212

      

URI,Short Name,Instance Count,Connectivity
http://schema.org/InteractionCounter,InteractionCounter,40525292,40487590
http://rdfs.org/sioc/ns#UserAccount,UserAccount,32477877,42183854
http://www.ics.forth.gr/isl/oae/core#Entity,Entity,27570278,27544245
http://rdfs.org/sioc/ns#Post,Post,20262647,0
http://www.gsi.dit.upm.es/ontologies/onyx/ns#EmotionSet,EmotionSet,20262646,20262646
http://rdfs.org/sioc/types#Tag,Tag,13804256,13790062
http://www.w3.org/2004/02/skos/core#Concept,Concept,306,590
http://gsi.dit.upm.es/ontologies/onyx/ns#EmotionCategory,EmotionCategory,291,578
http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat,QuadMapFormat,130,322
http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapFormat,array-of-QuadMapFormat,98,98


In [ ]:
show_class_list_as_html_table( ds2.create_list_of_classes(2,""), ds2.name, ds2.endpoint_url )

URI,Short Name,Instance Count,Connectivity
http://schema.org/InteractionCounter,InteractionCounter,40525292,40487590
http://rdfs.org/sioc/ns#UserAccount,UserAccount,32477877,42183854
http://www.ics.forth.gr/isl/oae/core#Entity,Entity,27570278,27544245
http://rdfs.org/sioc/ns#Post,Post,20262647,0
http://www.gsi.dit.upm.es/ontologies/onyx/ns#EmotionSet,EmotionSet,20262646,20262646
http://rdfs.org/sioc/types#Tag,Tag,13804256,13790062
http://www.w3.org/2004/02/skos/core#Concept,Concept,306,590
http://gsi.dit.upm.es/ontologies/onyx/ns#EmotionCategory,EmotionCategory,291,578
http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat,QuadMapFormat,130,322
http://www.openlinksw.com/schemas/virtrdf#array-of-QuadMapFormat,array-of-QuadMapFormat,98,98


In [ ]:
ds3 = DataSet("SIB COVID 19", "https://covid-19-sparql.expasy.org/sparql")
ds3.get_classes()
ds3.collect_connectivity()
# ds3.show_used_namespaces()
print()
# ds3.print_classes(10,"")
show_class_list_as_html_table( ds3.create_list_of_classes(10,""), ds3.name, ds3.endpoint_url )


      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://biohackathon.org/resource/faldo#ExactPosition>.
      }
      
10491650

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://biohackathon.org/resource/faldo#Region>.
      }
      
108449

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://biohackathon.org/resource/faldo#Position>.
      }
      
10403218

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://biohackathon.org/resource/vg#Step>.
      }
      
0

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://bioha

URI,Short Name,Instance Count,Connectivity
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,5281316,10491650
http://biohackathon.org/resource/faldo#Region,Region,5245908,108449
http://biohackathon.org/resource/faldo#Position,Position,5204334,10403218
http://biohackathon.org/resource/vg#Step,Step,5198168,0
http://biohackathon.org/resource/faldo#ForwardStrandPosition,ForwardStrandPosition,77084,88598
http://ddbj.nig.ac.jp/ontologies/nucleotide/Exon,Exon,39936,124238
http://ddbj.nig.ac.jp/ontologies/nucleotide/Mature_Peptide,Mature_Peptide,25161,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Coding_Sequence,Coding_Sequence,15820,0
http://ddbj.nig.ac.jp/ontologies/nucleotide/Protein,Protein,15820,15820
http://ddbj.nig.ac.jp/ontologies/nucleotide/Gene,Gene,14505,90402


In [ ]:
ds4 = DataSet("Covid 19 KG", "https://sparql.proconsortium.org/virtuoso5/sparql")
ds4.get_classes()
ds4.collect_connectivity()
# ds4.show_used_namespaces()
print()
# ds4.print_classes(10,"")
show_class_list_as_html_table( ds4.create_list_of_classes(10,""), ds4.name, ds4.endpoint_url )


      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <https://semrep.nlm.nih.gov/Entity>.
      }
      
38265866

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.w3.org/2002/07/owl#Axiom>.
      }
      
0

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <https://semrep.nlm.nih.gov/Relation>.
      }
      
2173171

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement>.
      }
      
36622

      PREFIX dbo: <http://dbpedia.org/ontology/>
      SELECT (COUNT(?expr) AS ?expr_COUNT) WHERE{
        ?expr ?p ?Drug.
        ?Drug a <http://biohackathon.org/resource/faldo#Exac

URI,Short Name,Instance Count,Connectivity
https://semrep.nlm.nih.gov/Entity,Entity,38265866,38265866
http://www.w3.org/2002/07/owl#Axiom,Axiom,11938498,0
https://semrep.nlm.nih.gov/Relation,Relation,2173171,2173171
http://www.w3.org/1999/02/22-rdf-syntax-ns#Statement,Statement,394660,36622
http://biohackathon.org/resource/faldo#ExactPosition,ExactPosition,240757,261782
http://biohackathon.org/resource/faldo#Position,Position,240757,261782
http://biohackathon.org/resource/faldo#Region,Region,130891,130939
http://purl.uniprot.org/core/Domain_Extent_Annotation,Domain_Extent_Annotation,49386,98772
http://purl.uniprot.org/core/Transmembrane_Annotation,Transmembrane_Annotation,45146,90292
http://purl.uniprot.org/core/Nucleotide_Resource,Nucleotide_Resource,35426,68967


In [ ]:

#Interesting classes from Covid on the web 
interesting_classes = ["Person",
                       "Place",
                       "Agent",
                       "Chemical", #"Chemical Substance"
                       "Disease",
                       "Procedure",
                       "Intervention",
                       "Claim",
                       "Outcome",
                       "Viruses",
                       "Biomolecule",
                       "Symptom",
                       "Chemical",
                       "Drugs",
                       "Mutation",
                       "Species",
                       "Device",
                       "Stain",
                       "Medication",
                       "Condition",
                       "Gene",
                       "Protein",
                       "Work",
                       "Position",
                       "Evidence"                    
                       ]

print("Searching for interesting classes in " + ds1.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds1.print_classes(1, c)
  print()



Searching for interesting classes in https://covidontheweb.inria.fr/sparql

Searching for Person
http://xmlns.com/foaf/0.1/Person	135616	Person		http://xmlns.com/foaf/0.1
http://ns.inria.fr/wasabi/ontology/Artist_Person	24264	Artist_Person		http://ns.inria.fr/wasabi/ontology
http://dbpedia.org/ontology/Person	5608	Person		http://dbpedia.org/ontology
http://dbpedia.org/resource/Person	203	Person		http://dbpedia.org/resource
http://dbpedia.org/ontology/MilitaryPerson	73	MilitaryPerson		http://dbpedia.org/ontology

Searching for Place
http://dbpedia.org/ontology/Place	6810	Place		http://dbpedia.org/ontology
http://dbpedia.org/ontology/PopulatedPlace	3835	PopulatedPlace		http://dbpedia.org/ontology
http://dbpedia.org/ontology/NaturalPlace	358	NaturalPlace		http://dbpedia.org/ontology
http://dbpedia.org/resource/Place	43	Place		http://dbpedia.org/resource
http://dbpedia.org/ontology/HistoricPlace	37	HistoricPlace		http://dbpedia.org/ontology

Searching for Agent
http://dbpedia.org/ontology/

In [ ]:
# interesting classes from Tweets KG
interesting_classes = ["Person", "Website", "Post", "UserAccount", "Disease", "ChemicalSubstance", "Agent", "Place" ]

print("Searching for interesting classes in " + ds2.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds2.print_classes(1, c)
  print()


Searching for interesting classes in https://data.gesis.org/tweetscov19/sparql

Searching for Person
http://dbpedia.org/class/yago/Person100007846	18	Person100007846		http://dbpedia.org/class/yago
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson	15	DUL.owl#NaturalPerson	NaturalPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://schema.org/Person	15	Person		http://schema.org
http://xmlns.com/foaf/0.1/Person	15	Person		http://xmlns.com/foaf/0.1
http://dbpedia.org/ontology/Person	15	Person		http://dbpedia.org/ontology
http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#SocialPerson	13	DUL.owl#SocialPerson	SocialPerson	http://www.ontologydesignpatterns.org/ont/dul/DUL.owl
http://dbpedia.org/ontology/PersonFunction	8	PersonFunction		http://dbpedia.org/ontology
http://umbel.org/umbel/rc/PersonWithOccupation	6	PersonWithOccupation		http://umbel.org/umbel/rc
http://dbpedia.org/class/yago/ReligiousPerson109628382	2	ReligiousPerson109628382		http://dbpedia.or

In [ ]:

# interesting classes from SIB Covid 19
interesting_classes = ["Gene", "Protein", "Work", "Position", "Evidence" ]

print("Searching for interesting classes in " + ds3.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://covid-19-sparql.expasy.org/sparql

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo

Searching for Evidence



In [ ]:
# interesting classes from Covid 19  KG
interesting_classes = ["Stain", "Strain", "Medication", "Condition", "Gene", "Protein", "Work", "Position" ]

print("Searching for interesting classes in " + ds4.endpoint_url)
print()

for c in interesting_classes:
  print("Searching for " + c )
  ds4.print_classes(1, c)
  print()

Searching for interesting classes in https://sparql.proconsortium.org/virtuoso5/sparql

Searching for Stain

Searching for Strain
http://purl.uniprot.org/core/Strain	33541	Strain		http://purl.uniprot.org/core

Searching for Medication

Searching for Condition

Searching for Gene
http://purl.uniprot.org/core/Gene	7031	Gene		http://purl.uniprot.org/core

Searching for Protein
http://purl.uniprot.org/core/Protein	7041	Protein		http://purl.uniprot.org/core

Searching for Work

Searching for Position
http://biohackathon.org/resource/faldo#ExactPosition	240757	faldo#ExactPosition	ExactPosition	http://biohackathon.org/resource/faldo
http://biohackathon.org/resource/faldo#Position	240757	faldo#Position	Position	http://biohackathon.org/resource/faldo



In [ ]:
# Classes that are present in more than one data set

#obtain list of all classes - take short class name not full url 

all_class_names = set()

datasets = [ds1, ds2, ds3, ds4]

for ds in datasets:
  for c in ds.classes:
    if len(c.short_name_fragment) > 0:
      all_class_names.add( c.short_name_fragment ) 
    else:
      all_class_names.add( c.short_name ) 

# print(all_class_names)

# check if this name is present in each of the datasets
# name is short name
name_2_presence_in_ds = {}
for name in all_class_names:
  # print (name)
  presence = [[]] * len(datasets)
  # print(presence)
  for i in range(0,len(datasets)):
    ds = datasets[i]
    list_of_cl = ds.get_classes_from_ds(name)
    if len(list_of_cl) > 0:
      presence[i] = [(cl.short_name, cl.instance_count) for cl in list_of_cl]

  # print(name + " " +str(presence))    
  name_2_presence_in_ds[name] = presence

print(len(name_2_presence_in_ds))

print(name_2_presence_in_ds)

  


def show_class_presence_in_ds_as_html(data, datasets):
  html = f"""<table border="1" >
            <caption> Class presence in datasets </caption>
            <thead>
            <tr>
              <th>Short Class Name</th>
        """
  for ds in datasets:
    html += f"<th>{ds.name}</th>\n"
              
  html += """
            </tr>              
            </thead>
          """
  for name in data:
      html += "<tr>"      
      html += "<td >%s</td>"%(name)
      x = data[name]
      # print(x)
      i=0
      for ds in datasets:
        if len(x[i]) > 0:
          html += "<td >"
          for e in x[i]:
            html+=e[0] + " : " + e[1] + "\n<br/>" 
          html += "</td>"
        else:
          html += f"<td ></td>"
        i+=1
        
      html += "</tr>"
  html += "</table>"
  display(HTML(html))
  print(data)
  

show_class_presence_in_ds_as_html(name_2_presence_in_ds, datasets)


1930
{'WikicatWoodworkers': [[], [('WikicatWoodworkers', '1')], [], []], 'Musician110339966': [[], [('Musician110339966', '1')], [], []], 'ChangeOfState100199130': [[], [('ChangeOfState100199130', '2')], [], []], 'Administrator109770949': [[], [('Administrator109770949', '4')], [], []], 'WikicatPeopleFromGaithersburg,Maryland': [[], [('WikicatPeopleFromGaithersburg,Maryland', '1')], [], []], 'Antagonist': [[('Antagonist', '24')], [], [], []], 'WikicatLiveTelevisionPrograms': [[], [('WikicatLiveTelevisionPrograms', '1')], [], []], 'WinterSportPlayer': [[('WinterSportPlayer', '40')], [], [], []], 'Politician110451263': [[], [('Politician110451263', '5')], [], []], 'Participant': [[], [('Participant110401829', '1'), ('WikicatParticipantsInAmericanRealityTelevisionSeries', '1')], [('Participant', '447')], [('Participant', '207')]], 'Theory': [[('Theory', '90')], [('Theory105989479', '1')], [], []], 'WikicatAmericanCivilRightsLawyers': [[], [('WikicatAmericanCivilRightsLawyers', '1')], [], 

Short Class Name,covid19ontheweb,tweetscov19,SIB COVID 19,Covid 19 KG
WikicatWoodworkers,,WikicatWoodworkers : 1,,
Musician110339966,,Musician110339966 : 1,,
ChangeOfState100199130,,ChangeOfState100199130 : 2,,
Administrator109770949,,Administrator109770949 : 4,,
"WikicatPeopleFromGaithersburg,Maryland",,"WikicatPeopleFromGaithersburg,Maryland : 1",,
Antagonist,Antagonist : 24,,,
WikicatLiveTelevisionPrograms,,WikicatLiveTelevisionPrograms : 1,,
WinterSportPlayer,WinterSportPlayer : 40,,,
Politician110451263,,Politician110451263 : 5,,
Participant,,Participant110401829 : 1 WikicatParticipantsInAmericanRealityTelevisionSeries : 1,Participant : 447,Participant : 207


{'WikicatWoodworkers': [[], [('WikicatWoodworkers', '1')], [], []], 'Musician110339966': [[], [('Musician110339966', '1')], [], []], 'ChangeOfState100199130': [[], [('ChangeOfState100199130', '2')], [], []], 'Administrator109770949': [[], [('Administrator109770949', '4')], [], []], 'WikicatPeopleFromGaithersburg,Maryland': [[], [('WikicatPeopleFromGaithersburg,Maryland', '1')], [], []], 'Antagonist': [[('Antagonist', '24')], [], [], []], 'WikicatLiveTelevisionPrograms': [[], [('WikicatLiveTelevisionPrograms', '1')], [], []], 'WinterSportPlayer': [[('WinterSportPlayer', '40')], [], [], []], 'Politician110451263': [[], [('Politician110451263', '5')], [], []], 'Participant': [[], [('Participant110401829', '1'), ('WikicatParticipantsInAmericanRealityTelevisionSeries', '1')], [('Participant', '447')], [('Participant', '207')]], 'Theory': [[('Theory', '90')], [('Theory105989479', '1')], [], []], 'WikicatAmericanCivilRightsLawyers': [[], [('WikicatAmericanCivilRightsLawyers', '1')], [], []], 

In [ ]:
sorted_by_presence = []
short_name_2_presence_and_instance_count = []
for short_name in name_2_presence_in_ds:
  # print(short_name)
  #number of datasets containing clases matching this short name
  presence_count = 0 
  curr_presence = name_2_presence_in_ds[short_name]
  # print("\t" + str(curr_presence))
  inst_count_list = [0]*4
  e_ind = 0
  for e in curr_presence:
    # print(e)
    if len(e) > 0:
      presence_count += 1
      
      z = 0
      for i in e:
        z += int(i[1])
      inst_count_list[e_ind]=z
    e_ind += 1

  short_name_2_presence_and_instance_count.append(
      (
          short_name, presence_count, 
          inst_count_list[0] if len(inst_count_list)>0 else 0,
          inst_count_list[1] ,
          inst_count_list[2] ,
          inst_count_list[3] ,
       )
  )

# print(short_name_2_presence_and_instance_count)

for i in sorted(short_name_2_presence_and_instance_count, reverse = True, key=lambda x: (x[1])):
  # if i[2] > 1 and i[3]>1 and i[4]>1 and i[5]>1:
  # if (i[2] + i[3] + i[4] + i[5]) > 1000 and "Person" in i[0]:
  if (i[2] + i[3] + i[4] + i[5]) > 10 and  i[1]>=3:
    print(i)
    # print(name_2_presence_in_ds[i[0]])
    # print()



# max_score = 0
# obj_with_max_score = None
# for e in name_2_presence_in_ds:
#   print(e + " " + str(name_2_presence_in_ds[e]))
#   curr_score = 0
#   for i in range(0, len(datasets)):
#     if name_2_presence_in_ds[e][i]:
#       curr_score += 1
#   if curr_score > max_score:
#     max_score = curr_score
#     obj_with_max_score = 
  

('Domain', 4, 34, 1, 187, 51988)
('array-of-QuadMap', 4, 98, 111, 111, 111)
('Function', 4, 5242, 8, 172, 8120)
('Property', 4, 39030, 40, 133, 235)
('array-of-QuadMapFormat', 4, 98, 98, 98, 98)
('Act', 4, 531, 88, 102, 22768)
('QuadMap', 4, 228, 265, 265, 265)
('Service', 4, 79, 5, 1, 7)
('Ontology', 4, 60, 6, 4, 7)
('Class', 4, 20335, 19, 134, 19691)
('Format', 4, 318, 230, 228, 228)
('Site', 4, 49, 10, 147, 2354)
('Part', 4, 726, 22, 509, 3758)
('Position', 4, 74347974, 2, 10562936, 481514)
('QuadMapFormat', 4, 228, 228, 228, 228)
('S', 4, 153654641, 20262954, 5306573, 477907)
('Name', 4, 945, 5, 483, 15757)
('Gene', 4, 90, 1, 15021, 7031)
('Region', 4, 983, 21, 5245972, 140429)
('Annotation', 4, 74801566, 5, 3382, 168168)
('Form', 4, 880, 238, 228, 228)
('ObjectProperty', 4, 4789, 3, 49, 6)
('Entity', 4, 74285546, 27570358, 735, 38265867)
('Structure', 4, 3433, 1, 796, 17280)
('Statement', 4, 62813, 3, 6227, 403435)
('Location', 4, 32, 24, 102, 6484)
('AnnotationProperty', 4, 469, 